In [77]:
import json
import os
import shapely
import folium
import folium.plugins
import re
import pandas as pd


In [7]:
## Import subway data
DATA_DIRECTORY = '/home/hugues/Dev/Opendata_Taipei/data'
stations_file = 'station_TRTC.json'
shapes_file = 'shape_TRTC.json'

In [11]:
with open(os.path.join(DATA_DIRECTORY, shapes_file)) as fp:
    shapes = json.load(fp)

with open(os.path.join(DATA_DIRECTORY, stations_file)) as fp:
    stations = json.load(fp)
print(f"{len(shapes)} lines and {len(stations)} stations loaded")

6 lines and 135 stations loaded


In [22]:
print('Example shapes')
print(shapes[0].keys())

print('Example stations')
print(stations[0].keys())

Example shapes
dict_keys(['LineNo', 'LineID', 'LineName', 'UpdateTime', 'Geometry'])
Example stations
dict_keys(['StationUID', 'StationID', 'StationName', 'StationAddress', 'BikeAllowOnHoliday', 'SrcUpdateTime', 'UpdateTime', 'VersionID', 'StationPosition', 'LocationCity', 'LocationCityCode', 'LocationTown', 'LocationTownCode'])


In [105]:
pattern = '([A-Z]{1,2})(\d+)'
prog = re.compile(pattern)
stations_lines = {s['StationID']:prog.search(s['StationID']).group(1) for s in stations}
COLORS = {'Y': '#fffc33',
         'BL':'#3371ff',
          'O':'#ffa833',
         'BR':'#75450c',
         'R':' #e81616',
         'G':' #4ca32b'}

In [81]:
map = folium.Map(location=[25.056468878784763, 121.51618034298974], zoom_start=12)

feature_shapes = folium.FeatureGroup(name='Lines').add_to(map)
feature_stations = folium.FeatureGroup(name='Stations').add_to(map)

for s in stations:
    s_line = stations_lines[s['StationID']]
    folium.Marker(location=[s['StationPosition']['PositionLat'], s['StationPosition']['PositionLon']],
                  popup=f"<i>{s['StationName']['En']}</i> {s_line}", 
                  icon=folium.plugins.BeautifyIcon(background_color=COLORS.get(s_line,'black'),
                                                   icon='fas fa-subway',
                                                   border_width=1,
                                                   prefix='fa')).add_to(feature_stations)
map

In [116]:
from shapely import wkt

for l in shapes:
    geom = wkt.loads(l['Geometry'])
    # color = {'fillColor':COLORS[l['LineID']], 'color':COLORS[l['LineID']]}
    folium.GeoJson(geom, 
                       style_function=lambda x, color=COLORS[l['LineID']]:{'color':color}).add_to(map)
map